# Data processing

In [1]:
import pandas as pd 
import glob 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os 
import sklearn
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
os.getcwd()

'/Users/julesbaudet/Documents/0. Cours/ENS/Deep Learning DIY/Projet final/DL_ens'

In [2]:
path = "/Users/julesbaudet/Documents/0. Cours/ENS/Deep Learning DIY/Projet final/ens_data"
# path = "/Users/linusbleistein/Documents/Cours ENS/Cours mathématiques/Deep learning 2020-2021/data_project"

In [3]:
frame = pd.read_csv(os.path.join(path,"clean_brave_data.csv"))
frame.head()

,Unnamed: 0,from_address,to_address,value,unix_block_timestamp,send_is_id,receive_is_id,labels_send,labels_receive
0,2438955,0x88e2efac3d2ef957fcd82ec201a506871ad06204,0x67fa2c06c9c6d4332f330e14a66bdf1873ef3d2b,1.0,1496084349,0,0,NaN,NaN
1,2305151,0x88e2efac3d2ef957fcd82ec201a506871ad06204,0x67fa2c06c9c6d4332f330e14a66bdf1873ef3d2b,7999999.0,1496085435,0,0,NaN,NaN
2,539714,0x88e2efac3d2ef957fcd82ec201a506871ad06204,0x67fa2c06c9c6d4332f330e14a66bdf1873ef3d2b,12000000.0,1496085651,0,0,NaN,NaN
3,539777,0x88e2efac3d2ef957fcd82ec201a506871ad06204,0x67fa2c06c9c6d4332f330e14a66bdf1873ef3d2b,113650000.0,1496085802,0,0,NaN,NaN
4,270439,0x00954e1c8fcf1c5c274aa10b1260a94564f47b58,0x023656f850bbf662e71006b3891e797653503286,10.0,1496241893,0,0,NaN,NaN


In [64]:
# df = frame.loc[(frame["send_is_id"]==1) | (frame["receive_is_id"]==1)]

In [5]:
df = frame

In [6]:
#%%time

#test1 = pd.crosstab(df['from_address'],df['to_address'])
# test1

In [7]:
%%time

from_c = CategoricalDtype(sorted(df.from_address.unique()), ordered=True)
to_c = CategoricalDtype(sorted(df.to_address.unique()), ordered=True)

row = df.from_address.astype(from_c).cat.codes
col = df.to_address.astype(to_c).cat.codes
sparse = csr_matrix((df["value"], (row, col)), \
                           shape=(from_c.categories.size, to_c.categories.size))
print(sparse.shape)

(717742, 952784)
CPU times: user 4.93 s, sys: 147 ms, total: 5.08 s
Wall time: 5.38 s


In [73]:
# import the known exchange list 
exch_labels = pd.read_csv(os.path.join(path, "exchanges_encoded.csv"), 
                          delimiter=';', names  = ['address','label']).set_index('address')
labels = exch_labels.to_dict()['label']
known_addresses = list(labels.keys())

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
from sklearn.decomposition import TruncatedSVD
#from scipy.sparse.linalg import svds

In [39]:
#%%time 
#u,s,vt = svds(sparse,100)

In [56]:
%%time
tsvd = TruncatedSVD(n_components=50, random_state=42)
out = tsvd.fit_transform(sparse)

CPU times: user 43.4 s, sys: 9.29 s, total: 52.7 s
Wall time: 43.7 s


In [57]:
tsvd_var_ratios = tsvd.explained_variance_ratio_
print(tsvd_var_ratios.sum())

0.9905612646937892


In [58]:
# taken on https://chrisalbon.com/machine_learning/feature_engineering/select_best_number_of_components_in_tsvd/
def select_n_components(var_ratio, goal_var: float) -> int:
    total_variance = 0.0
    n_components = 0
    for explained_variance in var_ratio:
        total_variance += explained_variance
        n_components += 1
        if total_variance >= goal_var:
            break
    return n_components

In [59]:
# Run function
select_n_components(tsvd_var_ratios, 0.99)

48

In [60]:
out.shape

(717742, 50)

In [77]:
data = pd.DataFrame(out)
data['label'] = np.isin(np.array(sorted(df.from_address.unique())),known_addresses).astype(int)

In [78]:
frame.to_csv(os.path.join(path,"processed_data.csv"))